# Generation of TX County based data for use with FIPS based geospatial mapping
This jupyter notebook describes the process to load assorted demographic data for TX and merge them into one table to be loaded for easy integrationg with FIPS county based time series data.

## Data Sources
1. Texas Public Health Region data from the "Center for Health Statistics Texas County Numbers and Public Health Regions" page at the [Texas Department of State Health Services][1]
2. US 2019 population from US Census website.  Excel downloaded and converted to County/State/FIPS [csv file][2] on personal git repo 
3. Household count estimates from the American Community Survey, 2018 5-year Data Profile 02. Data provided by Theresa Howard at UT CSR. ([file on github][3])
4. Final Compiled [Dataset][4] available on mepearson [tacc-dash repo][5]

[1]: https://dshs.texas.gov/chs/info/info_txco.shtm "DSHS"
[2]: https://github.com/mepearson/tacc_dash/blob/master/data/US_population.csv '2019 US Census Population Estimates'
[3]: https://github.com/mepearson/tacc_dash/blob/master/data/counties_ACS_DP02_5Y_2018_Households.csv 'ACS Household Estimates'
[4]: https://github.com/mepearson/tacc_dash/tree/master/data 'Compiled TX County Info'
[5]: https://github.com/mepearson/tacc_dash/tree/master/data 'mepearon tacc-dash repo' 

## Libraries

In [1]:
# Load Libraries
import os
from urllib.request import urlopen
import json
import pandas as pd
import plotly.express as px

## Get Data
### 1. Public Health Region Data

In [2]:
# TX health region link from https://dshs.texas.gov/chs/info/info_txco.shtm
healthregionurl =  'http://www.dshs.state.tx.us/chs/info/TxCoPhrMsa.xls'
txregions = pd.read_excel(healthregionurl)
# remove footer rows
txregions = txregions[0:254]
# Convert to state + county fips code
txregions['FIPS #'] = 48000 + txregions['FIPS #']
# save to local file and then upload to github
txregions.to_csv('dshs_county_info.csv')

### 2. US Population Data

In [3]:
# LOAD US POPULATION DATA
# # US Population downloaded from US Census
USpopfile = 'https://raw.githubusercontent.com/mepearson/tacc_dash/master/data/US_population.csv'
uspop = pd.read_csv(USpopfile)
# Convert population to numeric
uspop['POP'] = uspop['POP'].str.replace(',','')
uspop['Population'] = uspop['POP']
uspop['POP'] = pd.to_numeric(uspop['POP'])
# Split County, State column into separate columns
uspop[['County','State']] = uspop['CTY STATE'].str.split(', ',expand=True)
# Remove leading '.' from County name
uspop['County']  = uspop['County'] .str[1:]
# Remove ' County' from county name
uspop['County'] = uspop['County'].str.replace(' County','')
# Get TX specific population data
txdemo = uspop[uspop['State']=='Texas'][['State','County','Population']]

### 3. ACS Household Estimates

In [4]:
acsurl='https://raw.githubusercontent.com/mepearson/tacc_dash/master/data/counties_ACS_DP02_5Y_2018_Households.csv'
acs = pd.read_csv(acsurl)

In [5]:
#select relevant data columns and rename to match previous dataframes
acs = acs[['County Name', 'County FIPS', 'Estimate Total households','Margin of Error Total households']]

## Merge data sets

In [6]:
# Standardize column names as 'County' and 'FIPS'
txregions = txregions.rename(columns={"County Name": "County", "FIPS #": "FIPS"})
txdemo = txdemo.rename(columns={"FIPS #": "FIPS"})
acs = acs.rename(columns={"County Name":"County","County FIPS": "FIPS"})

In [7]:
# Merge into one tx_info dataset
tx_info = pd.merge(txregions, txdemo, on=['County'], how='outer')
tx_info = pd.merge(tx_info, acs, on=['County','FIPS'], how='outer')

In [8]:
### Export Dataset
tx_info.to_csv('compiled_tx_county_info.csv')

### Code to load compiled texas demographic data from github repository

In [9]:
txdataurl = 'https://raw.githubusercontent.com/mepearson/tacc_dash/master/data/compiled_tx_county_info.csv'
txdata = pd.read_csv(txdataurl,index_col=0)
txdata = txdata.rename(columns={'FIPS':'fips'})